### Revisión cifras para informe trimestral ONAVIS

In [1]:
##Borrar datos del entorno
rm(list=ls())

#Se utiliza acman para instalar y cargar paquetes
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse,janitor, httr,jsonlite)


Loading required package: pacman



In [2]:
#------------------ #Función para consultar la api

consulta_api<-function(url){
  r <- GET(url, timeout(10000))
  df <- fromJSON(rawToChar(r$content))
  return(df)
}


In [3]:
# Configura la opción de impresión para evitar notación científica y mostrar dos decimales
options(scipen = 999, digits = 2)

### Fovissste

In [5]:


urlfov<-"https://sniiv.sedatu.gob.mx/api/CuboAPI/GetFOVISSSTE/2024,2024/0/0/esquema"
fovissste<-consulta_api(urlfov)
str(fovissste)

unique_organismos <- unique(fovissste$esquema)
unique_organismos




'data.frame':	5 obs. of  4 variables:
 $ año     : int  0 0 0 0 0
 $ esquema : chr  "Tradicional" "Fovissste infonavit conyugal" "Pensionados" "Fovissste para todos" ...
 $ acciones: int  6595 274 60 560 50
 $ monto   : num  6245012939 210717451 35646423 103475332 40588965


[1] "Tradicional"                  "Fovissste infonavit conyugal"
[3] "Pensionados"                  "Fovissste para todos"        
[5] "Individual fov-inf"

In [13]:
fovissste %>%
  group_by(esquema)%>%
  select(acciones)%>%
  janitor::adorn_totals()

Adding missing grouping variables: `esquema`


esquema,acciones
<chr>,<int>
Tradicional,6595
Fovissste infonavit conyugal,274
Pensionados,60
Fovissste para todos,560
Individual fov-inf,50
Total,7539


### Insus  

In [7]:
urlinsus<-"https://sniiv.sedatu.gob.mx/api/CuboAPI/GetInsus/2024,2024/0/0/genero"
insus<-consulta_api(urlinsus)
str(insus)

'data.frame':	2 obs. of  4 variables:
 $ año     : int  0 0
 $ sexo    : chr  "Hombre" "Mujer"
 $ acciones: int  228 369
 $ monto   : int  3078000 4981500


In [8]:
insus %>%
  select(acciones) %>%
  slice(1:2) %>%
  summarise(total = sum(acciones))

total
<int>
597


### Sociedad Hipotecaria Federal

In [12]:
urlshf1<-"https://sniiv.sedatu.gob.mx/api/CuboAPI/GetFinanciamiento/2024,2024/0/0/organismo"
shf1<-consulta_api(urlshf1)
str(shf1)

'data.frame':	7 obs. of  4 variables:
 $ año      : int  0 0 0 0 0 0 0
 $ organismo: chr  "FOVISSSTE" "INFONAVIT" "CONAVI" "SHF (FONDEO)" ...
 $ acciones : int  7539 122916 70960 10718 73 17940 597
 $ monto    : num  6635441110 49122752565 2607253989 516392359 104228436 ...


In [14]:
 shf1 %>%
  filter(organismo == "SHF (FONDEO)") %>%
  select(acciones)

acciones
<int>
10718


### Infonavit

In [15]:
urlinfo<-"https://sniiv.sedatu.gob.mx/api/CuboAPI/GetINFONAVIT/2024,2024/0/0/linea_credito,modalidad,esquema"
infonavit<-consulta_api(urlinfo)
str(infonavit)

'data.frame':	16 obs. of  6 variables:
 $ linea_credito: chr  "Linea II" "Linea II" "Linea II" "Linea IV" ...
 $ año          : int  0 0 0 0 0 0 0 0 0 0 ...
 $ modalidad    : chr  "Nueva" "Existente" "Nueva" "Ampliación y rehabilitación" ...
 $ esquema      : chr  "Apoyo infonavit" "Cofinavit" "Cofinavit" "Mejoravit infonavit" ...
 $ acciones     : int  3579 960 3169 38327 218 2 2511 341 893 28366 ...
 $ monto        : num  0 792081203 2317690523 0 139267899 ...


In [16]:
unique_inf <- unique(infonavit$linea_credito)
unique_inf

[1] "Linea II"  "Linea IV"  "Linea V"   "Linea III"

In [17]:
unique_inf <- unique(infonavit$modalidad)
unique_inf

[1] "Nueva"                         "Existente"                    
[3] "Ampliación y rehabilitación"   "Pago de pasivos"              
[5] "Con disponibilidad de terreno" "Adquisición de suelo"         
[7] "Autoconstrucción"

In [25]:
unique_inf <- unique(infonavit$esquema)
unique_inf

[1] "Apoyo infonavit"     "Cofinavit"           "Mejoravit infonavit"
[4] "Tradicional"         "Infonavit total"     "ConstruYo infonavit"
[7] "INFONAVIT MejOraSí"

In [18]:
nuevaexist <- infonavit %>%
  filter(linea_credito == "Linea II") %>%
  filter(modalidad %in% c("Nueva", "Existente")) %>%
  group_by(esquema, modalidad) %>%
  summarise(acciones = sum(acciones)) %>%
  pivot_wider(names_from = modalidad, values_from = acciones) %>%
  mutate(Existente = replace_na(Existente, 0),
         Total = Nueva + Existente) %>%
  janitor::adorn_totals()

nuevaexist

`summarise()` has grouped output by 'esquema'. You can override using the
`.groups` argument.


esquema,Nueva,Existente,Total
<chr>,<int>,<int>,<int>
Apoyo infonavit,3579,0,3579
Cofinavit,3169,960,4129
Infonavit total,3846,1542,5388
Tradicional,27215,28366,55581
Total,37809,30868,68677


In [24]:
crediterreno <- infonavit %>%
  filter(linea_credito == "Linea II") %>%
  filter(modalidad %in% c("Adquisición de suelo")) %>%
  select(acciones)
crediterreno

acciones
<int>
893


In [30]:
linea_3 <- infonavit %>%
  filter(linea_credito == "Linea III") %>%
  filter(esquema %in% c("ConstruYo infonavit", "Tradicional")) %>%

  summarise(acciones = sum(acciones))

linea_3

acciones
<int>
2858


In [32]:
linea_4 <- infonavit %>%
  filter(linea_credito == "Linea IV") %>%
  filter(modalidad %in% c("Ampliación y rehabilitación", "Autoconstrucción")) %>%
  group_by(esquema) %>%
  summarise(acciones = sum(acciones))%>%
janitor::adorn_totals()
linea_4

esquema,acciones
<chr>,<int>
ConstruYo infonavit,86
INFONAVIT MejOraSí,11855
Mejoravit infonavit,38327
Total,50268


In [33]:
linea_5 <- infonavit %>%
  filter(linea_credito == "Linea V") %>%
  group_by(esquema) %>%
  summarise(acciones = sum(acciones)) %>%
  adorn_totals()

linea_5

esquema,acciones
<chr>,<int>
Tradicional,220
Total,220


### Conavi

In [35]:

url<-"https://sniiv.sedatu.gob.mx/api/CuboAPI/GetCONAVI/2024,2024/0/0/programa_presupuestal,modalidad,mes"
conavi<-consulta_api(url)
str(conavi)

'data.frame':	6 obs. of  6 variables:
 $ año                  : int  0 0 0 0 0 0
 $ mes                  : chr  "febrero" "abril" "abril" "marzo" ...
 $ modalidad            : chr  "Mejoramiento" "Mejoramiento" "Reconstrucción" "Vivienda nueva" ...
 $ programa_presupuestal: chr  "Programa de Vivienda Social (Subsidio 100% CONAVI)" "Programa de Vivienda Social (Subsidio 100% CONAVI)" "Programa de Vivienda Social (Subsidio 100% CONAVI)" "Programa de Vivienda Social (Subsidio 100% CONAVI)" ...
 $ acciones             : int  70620 6 1605 340 1183 6
 $ monto                : num  2471700000 474855 141591524 135553989 251269440 ...


In [36]:
unique_conavi <- unique(conavi$programa_presupuestal)
unique_conavi

[1] "Programa de Vivienda Social (Subsidio 100% CONAVI)"

In [39]:

unique_conavi <- unique(conavi$modalidad)
unique_conavi

[1] "Mejoramiento"   "Reconstrucción" "Vivienda nueva" "Ampliación"

In [38]:
conavi1 <- conavi %>%
  filter(modalidad %in% c("Mejoramiento", "Vivienda nueva")) %>%
  filter(mes %in% c("febrero", "marzo")) %>%
  group_by(programa_presupuestal) %>%
  select(acciones)%>%
  janitor::adorn_totals()

conavi1

Adding missing grouping variables: `programa_presupuestal`


programa_presupuestal,acciones
<chr>,<int>
Programa de Vivienda Social (Subsidio 100% CONAVI),70620
Programa de Vivienda Social (Subsidio 100% CONAVI),340
Total,70960


In [50]:
#------------------ CONAVI SIESCO

url<-"https://siesco.conavi.gob.mx/SNIIV.svc/getCONAVI/2024,2024/0/0/programa_presupuestal,modalidad_anterior,mes"
conavi<-consulta_api(url)
str(conavi)

'data.frame':	6 obs. of  6 variables:
 $ acciones             : int  70620 6 1605 340 1183 6
 $ año                  : int  0 0 0 0 0 0
 $ mes                  : chr  "febrero" "abril" "abril" "marzo" ...
 $ modalidad_anterior   : chr  "Mejoramiento" "Mejoramiento" "Reconstrucción" "Vivienda nueva" ...
 $ monto                : num  2471700000 474855 141591524 135553989 251269440 ...
 $ programa_presupuestal: chr  "Programa de Vivienda Social (Subsidio 100% Conavi)" "Programa de Vivienda Social (Subsidio 100% Conavi)" "Programa de Vivienda Social (Subsidio 100% Conavi)" "Programa de Vivienda Social (Subsidio 100% Conavi)" ...


In [51]:
unique_conavi <- unique(conavi$programa_presupuestal)
unique_conavi

[1] "Programa de Vivienda Social (Subsidio 100% Conavi)"

In [54]:
unique_conavi <- unique(conavi$modalidad_anterior)
unique_conavi

[1] "Mejoramiento"   "Reconstrucción" "Vivienda nueva" "Ampliación"

In [55]:
siesco <- conavi %>%
  filter(modalidad_anterior %in% c("Mejoramiento", "Vivienda nueva")) %>%
  filter(mes %in% c("febrero", "marzo")) %>%
  group_by(programa_presupuestal) %>%
  select(acciones)%>%
  janitor::adorn_totals()

siesco

Adding missing grouping variables: `programa_presupuestal`


programa_presupuestal,acciones
<chr>,<int>
Programa de Vivienda Social (Subsidio 100% Conavi),70620
Programa de Vivienda Social (Subsidio 100% Conavi),340
Total,70960
